In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {device}")


train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(15),  
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),  
    transforms.ColorJitter(brightness=0.3, contrast=0.3),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder("dataset/train", transform=train_transforms)
val_dataset = ImageFolder("dataset/val", transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Классы: {train_dataset.classes}")


Используем устройство: cpu
Классы: ['defect', 'no-defect']


In [ ]:
import torch.nn as nn
from torchvision import models


model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Модель подготовлена к обучению")


c:\Users\Alexandr\Documents\Варя\ATP\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Alexandr\Documents\Варя\ATP\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Модель подготовлена к обучению


In [ ]:
from torch import save


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):

    best_accuracy = 0.0
    for epoch in range(num_epochs):
        print(f"Эпоха {epoch+1}/{num_epochs}")
        
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Средняя потеря на обучении: {running_loss / len(train_loader)}")

        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Точность на валидации: {accuracy:.2f}%")
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            save(model.state_dict(), "defect_model.pth")
            print("Сохранена модель с аккуратностью: ", accuracy)

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)


Эпоха 1/20
Средняя потеря на обучении: 0.4148086006087916
Точность на валидации: 71.43%
Сохранена модель с аккуратностью:  71.42857142857143
Эпоха 2/20
Средняя потеря на обучении: 0.14780109801462718
Точность на валидации: 76.79%
Сохранена модель с аккуратностью:  76.78571428571429
Эпоха 3/20
Средняя потеря на обучении: 0.10934355628809758
Точность на валидации: 82.14%
Сохранена модель с аккуратностью:  82.14285714285714
Эпоха 4/20
Средняя потеря на обучении: 0.08100994962400623
Точность на валидации: 71.43%
Эпоха 5/20
Средняя потеря на обучении: 0.06957430505592908
Точность на валидации: 85.71%
Сохранена модель с аккуратностью:  85.71428571428571
Эпоха 6/20
Средняя потеря на обучении: 0.06897162819015128
Точность на валидации: 87.50%
Сохранена модель с аккуратностью:  87.5
Эпоха 7/20
Средняя потеря на обучении: 0.04700993886217475
Точность на валидации: 87.50%
Эпоха 8/20
Средняя потеря на обучении: 0.05627946530668331
Точность на валидации: 85.71%
Эпоха 9/20
Средняя потеря на обучении

In [4]:
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Точность на валидации: {accuracy:.2f}%")

Точность на валидации: 87.50%


In [ ]:
import cv2
import numpy as np
import os

def remove_red_triangle(image_path, save_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка загрузки {image_path}")
        return

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_red1 = np.array([0, 120, 70])   
    upper_red1 = np.array([10, 255, 255]) 
    lower_red2 = np.array([170, 120, 70]) 
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 + mask2  

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        roi = image[y:y+h, x:x+w] 
        blurred = cv2.GaussianBlur(roi, (15, 15), 0)  
        image[y:y+h, x:x+w] = blurred  

    cv2.imwrite(save_path, image)
    print(f"Обработанное изображение сохранено в {save_path}")

remove_red_triangle("dataset/val/defect/photo_2025-02-02_18-47-09.jpg", "new_photo.jpg")

Ошибка загрузки dataset/val/defect/photo_2025-02-02_18-47-09.jpg
